# Advanced Querying Mongo

Importing libraries and setting up connection

In [55]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [56]:
#reaching through the database
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [57]:
#conecting to the correct database
db = client.get_database("Ironhack")
db.list_collection_names()

['restaurants', 'Companies']

In [58]:
#chosing the correct colecction
collection = db.get_collection("Companies")
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'Companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [65]:
# Your Code
filtro = {"name": "Babelgum"}
proj = {"_id":0, "name":1}
babelgum = list(collection.find(filtro, proj))
babelgum

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [78]:
# Your Code
proj = {"_id":0,"name":1,"number_of_employees":1}
n_employees = list(collection.find({"number_of_employees": {"$gt": 5000}},proj).limit(20).sort("number_of_employees",-1))
n_employees

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [86]:
# Your Code
query = {"$and": [{"founded_year": {"$gte":2000}},{"founded_year":{"$lte":2005}}]}
proj = {"_id":0,"name":1,"founded_year":1}
founded_2000_2005 = list(collection.find(query,proj).limit(20))
founded_2000_2005

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [89]:
# Your Code
query = {"$and": [{"ipo.valuation_amount":{"$gte":100_000_000}},{"founded_year": {"$lt":2010}}]}
proj = {"_id":0,"name":1,"ipo":1}
campanies_100M = list(collection.find(query,proj).limit(3))
campanies_100M

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [92]:
# Your Code
query = {"$and": [{"number_of_employees":{"$lt":1000}},{"founded_year": {"$lt":2005}}]}
proj = {"_id":0,"name":1,"number_of_employees":1, "founded_year": 1}
exercise_5 = list(collection.find(query,proj).limit(10).sort("number_of_employees",-1))
exercise_5

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Buongiorno', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'Cvent', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000}]

### 6. All the companies that don't include the `partners` field.

In [93]:
# Your Code
list(collection.find({"partners": {"$exists": False}}))


[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [99]:
# Your Code
proj = {"_id":0,"name":1}
exercise_7 = list(collection.find({"category_code": "null"},proj))
exercise_7

[]

In [100]:
exercise_7 = list(collection.find({"category_code": " "},proj))
exercise_7

[]

In [103]:
exercise_7 = list(collection.find({"category_code": None},proj).limit(10))
exercise_7

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [110]:
# Your Code
query = {"$and": [{"number_of_employees":{"$gte":100}},{"number_of_employees":{"$lt":1000}}]}
proj = {"_id":0,"name":1,"number_of_employees":1}
exercise_8 = list(collection.find(query,proj).limit(20))
exercise_8

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [118]:
# Your Code
proj = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}
query = {"ipo.valuation_amount": {"$gte":0}}
list(collection.find(query,proj).limit(10).sort("ipo.valuation_amount",-1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [120]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
query = {"number_of_employees": {"$gt":0}}
list(collection.find(query,proj).limit(10).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [122]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_month": 1}
query = {"founded_month": {"$gte":6}}
list(collection.find(query,proj).limit(10))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [124]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_month": 1}
query = {"$and": [{"founded_year": {"$lt": 2000}},{"acquisition.price_amount": {"$gt":10_000_000}}]}
list(collection.find(query,proj).limit(10))

[{'name': 'Postini', 'founded_month': 6},
 {'name': 'SideStep', 'founded_month': 11},
 {'name': 'Recipezaar', 'founded_month': 9},
 {'name': 'PayPal', 'founded_month': 12},
 {'name': 'Snapfish', 'founded_month': None},
 {'name': 'Neopets', 'founded_month': 11},
 {'name': 'Sun Microsystems', 'founded_month': 2},
 {'name': 'Zappos', 'founded_month': 1},
 {'name': 'Alibaba', 'founded_month': 6},
 {'name': 'Sabre', 'founded_month': None}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [128]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
query = {"acquisition.acquired_year": {"$gt":2010}}
list(collection.find(query,proj).limit(3).sort("acquisition.price_amount"))

[{'name': 'blogTV',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2013/03/13/younow-buys-blogtv/',
   'source_description': 'Live Social Video Network YouNow Acquires Streaming Service BlogTV',
   'acquired_year': 2013,
   'acquired_month': 3,
   'acquired_day': 13,
   'acquiring_company': {'name': 'YouNow', 'permalink': 'younow'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [129]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_year": 1}
query = {"founded_year": {"$gt":0}}
list(collection.find(query,proj).limit(10).sort("founded_year"))

[{'name': 'US Army', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'VideoSurf', 'founded_year': 1840}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [135]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_day": 1}
query = {"$and":[{"founded_day": {"$gte":0}},{"founded_day": {"$lte":7}}]}
list(collection.find(query,proj).limit(10).sort("acquisition.price_amount",-1))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [136]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
query = {"$and":[{"category_code":"web"},{"number_of_employees": {"$gt": 4000}}]}
list(collection.find(query,proj).limit(10).sort("number_of_employees",1))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [139]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}
query = {"$and":[{"acquisition.price_amount":{"$gt": 10_000_000}},{"acquisition.price_currency_code":"EUR"}]}
list(collection.find(query,proj))

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}},
 {'name': 'Tuenti Technologies', 'acquisition': {'price_amount': 70000000}},
 {'name': 'BioMed Central', 'acquisition': {'price_amount': 43400000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [141]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
query = {"acquisition.acquired_month":{"$in": [1,2,3]}}
list(collection.find(query,proj).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [154]:
# Your Code
proj = {"_id": 0, "name": 1,"founded_year":1,"acquisition.acquired_year":1 }
query = {"and":[{"founded_year":{"$gte": 2000}},{"founded_year":{"$lte": 2010}}, {"acquisition.acquired_year":{"$gt":2011}}]}
list(collection.find(query,proj).limit(10))


[]

### 20. All the companies that have been 'deadpooled' after the third year.

In [143]:
# Your Code
proj = {"_id": 0, "name": 1,"founded_year":1, "deadpooled_year": 1}
query = {"deadpooled_year":{"$gt":3}}
list(collection.find(query,proj).limit(10))

[{'name': 'Omnidrive', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'founded_year': 2007, 'deadpooled_year': 2013},
 {'name': 'Sparter', 'founded_year': 2007, 'deadpooled_year': 2008},
 {'name': 'Thoof', 'founded_year': 2006, 'deadpooled_year': 2013},
 {'name': 'Mercora', 'founded_year': None, 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'founded_year': 2005, 'deadpooled_year': 2010},
 {'name': 'Jangl SMS', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Stickam', 'founded_year': 2006, 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'founded_year': 2004, 'deadpooled_year': 2008},
 {'name': 'EQO', 'founded_year': 2006, 'deadpooled_year': 2012}]